In [3]:
# import libraries
import nltk
import sys
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

import re
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import pickle


sys.path.append("./models")
from starting_verb_extractor import StartingVerbExtractor

from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from sklearn.base import BaseEstimator, TransformerMixin


[nltk_data] Downloading package punkt to /home/fuh/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/fuh/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/fuh/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [4]:
# load data from database
engine = create_engine('sqlite:///data/DisasterResponse.db')
df = pd.read_sql_table('message_and_category', engine)  
#drop child_alone category that has only zeros. Otherwise SGDClassifier will not be able to run.
df = df.drop(['child_alone'], axis=1)
X = df.message
Y = df[df.columns[4:]]
labels = df[df.columns[4:]].columns


In [5]:
df[df.columns[4:]]

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26211,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26212,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26213,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26214,1,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [6]:
# url Regular expression
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

def tokenizer(text):
    
    # Get all url's in text
    detected_urls = re.findall(url_regex, text)
    
    # replace all urls in text with 'urlplaceholder'
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    # Tokenize 'text' and return tokenized array
    tokens = word_tokenize(text)
    
    # Initialize Lemmatizer, to reduce all words to thier normal form
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    
    # Go through token to to lammatize, normalize and strip all white spaces
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
# Cuustom Estimator 'StartingVerbExtractor' that extracts the starting verb of a sentence
class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    '''
    Description: Custom Estimator 'StartingVerbExtractor' that extracts the starting verb of a sentence
    
    Baseclasses:
        BaseEstimator
        TransformerMixin
    '''
    
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
        

    def starting_verb(self, text):
        # Extract sentences from text
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            # Extract words into seperate tags
            pos_tags = nltk.pos_tag(self.tokenizer(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged) 
    

In [8]:
pipeline  = Pipeline([
        ('features', FeatureUnion([ # 

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenizer)), # Generate bag of words bag given document
                ('tfidf', TfidfTransformer()) # Gives weighting to words matrix from previous estimator
            ])),

            ('starting_verb', StartingVerbExtractor(tokenizer=tokenizer)) #Extract the starting verbs of a sentence 
        ])),

        ('clf', MultiOutputClassifier(SGDClassifier(random_state=42)))
    ])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

model = pipeline
model.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('features',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('text_pipeline',
                                                 Pipeline(memory=None,
                                                          steps=[('vect',
                                                                  CountVectorizer(analyzer='word',
                                                                                  binary=False,
                                                                                  decode_error='strict',
                                                                                  dtype=<class 'numpy.int64'>,
                                                                                  encoding='utf-8',
                                                                                  input='content',
                                                                                  low

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
y_pred = model.predict(X_test)

In [13]:
(y_pred == y_test).mean().mean()

0.9488120667858234

In [208]:

def display_results(model, y_test, y_pred, has_params=False):
    '''Display Report of training '''
    
    labels = df[df.columns[4:]].columns
    print("Labels:", labels)
    
#     accuracy = (y_pred == y_test).mean()
#     print("Accuracy:", accuracy)
    
    if has_params: #If params are used display best params an best scores
        print("\nBest Parameters:", model.best_params_)
        print("\nBest Scores:",model.best_score_)
    
    for i, column in enumerate(y_test.columns):
        print('Model Performance with Category: {}'.format(column))
        print('Accuracy for {}: {}'.format(column, accuracy_score(y_test[column],  y_pred.T[i][:])))
        print(classification_report(np.array(y_test[column]),y_pred.T[i])) # Transpose y to 
        
#     for i, col in enumerate(columns): #Go through colouns 
#         print(i, col)
#         print(classification_report(y_test.to_numpy()[:1],  y_pred[:1], target_names=columns))
#         print(accuracy_score(y_test.to_numpy()[:, 1],  y_pred[:, 1]))

display_results(model, y_test, y_pred)

Labels: Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'water', 'food', 'shelter', 'clothing', 'money', 'missing_people',
       'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport',
       'buildings', 'electricity', 'tools', 'hospitals', 'shops',
       'aid_centers', 'other_infrastructure', 'weather_related', 'floods',
       'storm', 'fire', 'earthquake', 'cold', 'other_weather',
       'direct_report'],
      dtype='object')
Model Performance with Category: related
Accuracy for related: 0.820415013732072
              precision    recall  f1-score   support

           0       0.74      0.43      0.55      1572
           1       0.83      0.95      0.89      4931
           2       1.00      0.06      0.11        51

    accuracy                           0.82      6554
   macro avg       0.86      0.48      0.52      6554
weighted avg       0.81      0.82      

/home/fuh/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy for medical_help: 0.9295086969789441
              precision    recall  f1-score   support

           0       0.93      0.99      0.96      6057
           1       0.64      0.16      0.25       497

    accuracy                           0.93      6554
   macro avg       0.79      0.57      0.61      6554
weighted avg       0.91      0.93      0.91      6554

Model Performance with Category: medical_products
Accuracy for medical_products: 0.9606347268843455
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6241
           1       0.78      0.24      0.37       313

    accuracy                           0.96      6554
   macro avg       0.87      0.62      0.68      6554
weighted avg       0.95      0.96      0.95      6554

Model Performance with Category: search_and_rescue
Accuracy for search_and_rescue: 0.9746719560573696
              precision    recall  f1-score   support

           0       0.98      1.00      0.99

### 6. Improve your model
Use grid search to find better parameters. 

In [209]:
parameters = {
    'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
    'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
}

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=3)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

model = cv
model.fit(X_train, y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV] features__text_pipeline__vect__max_df=0.5, features__text_pipeline__vect__ngram_range=(1, 1) 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  features__text_pipeline__vect__max_df=0.5, features__text_pipeline__vect__ngram_range=(1, 1), score=0.290, total= 2.4min
[CV] features__text_pipeline__vect__max_df=0.5, features__text_pipeline__vect__ngram_range=(1, 1) 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.4min remaining:    0.0s


[CV]  features__text_pipeline__vect__max_df=0.5, features__text_pipeline__vect__ngram_range=(1, 1), score=0.290, total= 2.4min
[CV] features__text_pipeline__vect__max_df=0.5, features__text_pipeline__vect__ngram_range=(1, 1) 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  4.8min remaining:    0.0s


[CV]  features__text_pipeline__vect__max_df=0.5, features__text_pipeline__vect__ngram_range=(1, 1), score=0.292, total= 2.4min
[CV] features__text_pipeline__vect__max_df=0.5, features__text_pipeline__vect__ngram_range=(1, 1) 
[CV]  features__text_pipeline__vect__max_df=0.5, features__text_pipeline__vect__ngram_range=(1, 1), score=0.292, total= 2.4min
[CV] features__text_pipeline__vect__max_df=0.5, features__text_pipeline__vect__ngram_range=(1, 1) 


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
pipeline_clf_acabooster  = Pipeline([
        ('features', FeatureUnion([ # 

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)), # Generate bag of words bag given document
                ('tfidf', TfidfTransformer()) # Gives weighting to words matrix from previous estimator
            ])),

            ('starting_verb', StartingVerbExtractor()) #Extract the starting verbs of a sentence 
        ])),

        ('clf', MultiOutputClassifier(SGDClassifier(random_state=42)))
    ])

X_train, X_test, y_train, y_test = train_test_split(X, Y)

model = pipeline_clf_acabooster
model.fit(X_train, y_train)

# Note: This is just a demonstration of using another algorithm, but not the later has a higher score

### 9. Export your model as a pickle file

In [ ]:
pickle_filepath = 'models/classifier.pkl'

pickle.dump(model, open(pickle_filepath, 'wb'))
print("Model Saved to: {}".format(pickle_filepath))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.